## Clone dataset (for Colab)

In [ ]:
!git clone https://github.com/mkecera/InstColorization.git
%cd InstColorization

## Initialize hyperparameters

- `TRAIN_DATASET_DIR` directory of the training dataset (will be downloaded)
- `VAL_DATASET_DIR` directory of the validation dataset (will be downloaded)
- `LEARNING_RATE` the learning rate
- `EPOCHS` the amount of epochs
- `EXPERIMENT_NAME` the label of the experiment. It is used in order to label the result files

In [ ]:
TRAIN_DATASET_DIR="./train_tiny"
VAL_DATASET_DIR="./val_tiny"

LEARNING_RATE = 0.0005
EPOCHS = 20
EXPERIMENT_NAME="larger"

In [ ]:
!mkdir -p loss_results
!mkdir -p plot_results

## Get training dataset

Downloads the training dataset from amazon aws.

In [ ]:
!wget "https://gatech-deep-learning.s3.eu-west-1.amazonaws.com/train_tiny.zip"
!unzip train_tiny.zip
!rm train_tiny.zip

## Get validation dataset

Downloads the validation dataset from amazon aws.

In [ ]:
!wget "https://gatech-deep-learning.s3.eu-west-1.amazonaws.com/val_tiny.zip"
!unzip val_tiny.zip
!rm val_tiny.zip


## Get testing dataset

Downloads the testing dataset from amazon aws.

In [ ]:
!wget "https://gatech-deep-learning.s3.eu-west-1.amazonaws.com/test_tiny.zip"
!unzip test_tiny.zip
!rm test_tiny.zip


## Install Conda

Conda is used as dependency manager

In [ ]:
# Install conda and add channels to look for packages in
import sys
! wget https://repo.anaconda.com/archive/Anaconda3-2020.02-Linux-x86_64.sh
! chmod +x Anaconda3-2020.02-Linux-x86_64.sh
! bash ./Anaconda3-2020.02-Linux-x86_64.sh -b -f -p /usr/local
sys.path.append('/usr/local/lib/python3.7/site-packages/')
! conda update -n base -c defaults conda -y
! conda config --add channels bioconda
! conda config --add channels conda-forge

## Install dependencies

In [ ]:
!conda env create --file env.yml
# !conda remove --name instacolorization --all

In [ ]:
!source activate instacolorization && conda env list

In [ ]:
!sh scripts/install.sh
!pip install opencv-python visdom gdown

## Download checkpoints

In [ ]:
!rm -r checkpoints/
!gdown --id 1Xb-DKAA9ibCVLqm8teKd1MWk6imjwTBh
!unzip checkpoints.zip

## Prepare bounding boxes

In [ ]:
!python inference_bbox.py --test_img_dir $TRAIN_DATASET_DIR --filter_no_obj
!python inference_bbox.py --test_img_dir $VAL_DATASET_DIR --filter_no_obj

## Train and validate third stage - Fusion module

In [ ]:
!mkdir ./checkpoints/$EXPERIMENT_NAME
!cp ./checkpoints/coco_finetuned_mask_256/latest_net_G.pth ./checkpoints/$EXPERIMENT_NAME/latest_net_GF.pth
!cp ./checkpoints/coco_finetuned_mask_256/latest_net_G.pth ./checkpoints/$EXPERIMENT_NAME/latest_net_G.pth
!cp ./checkpoints/coco_finetuned_mask_256/latest_net_GComp.pth ./checkpoints/$EXPERIMENT_NAME/latest_net_GComp.pth

!python train_with_validation.py --stage fusion --fusion_type larger --name $EXPERIMENT_NAME  --train_img_dir train_tiny --test_img_dir val_tiny --sample_p 1.0 --niter $EPOCHS --niter_decay 0 --lr $LEARNING_RATE --model train --load_model --display_ncols 4 --fineSize 256 --batch_size 1 --display_freq 1 --print_freq 1 --save_epoch_freq 10 --no_html --display_id -1

## Download plots

In [ ]:
!zip -r plot_results.zip ./plot_results/

from google.colab import files
files.download("plot_results.zip")